#발표에 반드시 포함되어야 하는 내용은 다음과 같습니다.

##1. **지역에 따라서** 선호하는 **게임 장르**가 다를까 라는 질문에 대답을 하셔야합니다.
##2. **연도별 게임의 트렌드**가 있을까 라는 질문에 대답을 하셔야합니다.
##3. **출고량이 높은 게임**에 대한 **분석 및 시각화 프로세스**가 포함되어야 합니다.

위 3가지 내용과 여러분들의 개인적인 분석 목표, 질문을 더하여 프로젝트를 완성하세요.

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import PCA
from scipy import stats
import matplotlib.pyplot as plt
import datetime
from scipy.stats import chisquare

In [ ]:
file = '/gdrive/My Drive/PROJECT_section1/vgames2.csv'
vgm = pd.read_csv(file, index_col = 0 )

In [ ]:
vgm.head(20)

In [ ]:
#각 구역 결측치 대체
vgm['Year'].fillna('0000', inplace=True)
vgm['Genre'].fillna('Unknown', inplace=True)
vgm['Publisher'].fillna('Unknown', inplace=True)


# Year/Genre/Publisher 결측치 가진 항목 중 같은 게임 다른 플랫폼 게임에서 데이터 따와서 결측치 최대한 제거

In [ ]:
# 결측치가 있는 row만 따로 모은다.
vgm_null_year = pd.DataFrame(vgm[vgm['Year']== '0000'])
vgm_null_gen = pd.DataFrame(vgm[vgm['Genre']=='Unknown'])
vgm_null_pub = pd.DataFrame(vgm[vgm['Publisher']=='Unknown'])

In [ ]:
#따로 모은 DF의 각 게임의 이름을 전체 DF에서 찾아 만약 데이터가 있으면 복사.

for i in range(len(vgm_null_year)) :
    x = pd.Series(vgm_null_year.iloc[i])
    for j in range(len(vgm[vgm['Name'] == x[0]].index)):
        y = pd.Series(vgm[vgm['Name'] == x[0]].iloc[j])
        if y[2] != '0000' and x[2] == '0000' :
            x[2] = y[2]
            vgm_null_year.iloc[i] = x

for i in range(len(vgm_null_gen)) :
    x = pd.Series(vgm_null_gen.iloc[i])
    for j in range(len(vgm[vgm['Name'] == x[0]].index)):
        y = pd.Series(vgm[vgm['Name'] == x[0]].iloc[j])
        if y[3] != 'Unknown' and x[3] == 'Unknown' :
            x[3] = y[3]
            vgm_null_gen.iloc[i] = x


for i in range(len(vgm_null_pub)) :
    x = pd.Series(vgm_null_pub.iloc[i])
    for j in range(len(vgm[vgm['Name'] == x[0]].index)):
        y = pd.Series(vgm[vgm['Name'] == x[0]].iloc[j])
        if y[4] != 'Unknown' and x[4] == 'Unknown' :
            x[4] = y[4]
            vgm_null_pub.iloc[i] = x

In [ ]:
#복사되어 데이터가 있는 vgm_null_XXX를 기존 DF에 복사해서 넣기
for i in vgm_null_year.index :
    vgm.loc[i] = pd.Series(vgm_null_year.loc[i])

for i in vgm_null_gen.index :
    vgm.loc[i] = pd.Series(vgm_null_gen.loc[i])

for i in vgm_null_pub.index :
    vgm.loc[i] = pd.Series(vgm_null_pub.loc[i])


In [ ]:
# Year - string으로 변환
def toStr(x) :    
    x = str(x)[:4]
    x = x.split('.')[0]
    return x

# Year - date형(int?)으로 전환
def toDate(x) : 
    if len(x) == 4 and x !='0000'  :
        return datetime.datetime.strptime(x, '%Y').year
    elif x == '0000' :
        return int(x)
    elif len(x) == 2 :
        if int(x[0]) <= 2:
            x = '20' + x
            return datetime.datetime.strptime(x, '%Y').year       
        else :
            x = '19'+x
            return datetime.datetime.strptime(x, '%Y').year
    else :
        x= '200'+x
        return datetime.datetime.strptime(x, '%Y').year

        

vgm['Year'] = vgm['Year'].apply(toStr)
vgm['Year'] = vgm['Year'].apply(toDate)
# vgm['Year'].value_counts()

In [ ]:
#판매량 수치 조절 / M은 걍 지워주고, K는 지우고 0.001 곱해준다
def uniform(x):
    if "M" in x :
        x= x.replace('M','')
        return  float(x)
    elif "K" in x :
        x = x.replace('K','')
        return 0.001 * float(x)
    else :
        return float(x)
    
vgm['NA_Sales'] = vgm['NA_Sales'].apply(uniform)
vgm['EU_Sales'] = vgm['EU_Sales'].apply(uniform)
vgm['JP_Sales'] = vgm['JP_Sales'].apply(uniform)
vgm['Other_Sales'] = vgm['Other_Sales'].apply(uniform)


In [ ]:
vgm_null_year1 = pd.DataFrame(vgm[vgm['Year']== 0])
vgm_null_gen1 = pd.DataFrame(vgm[vgm['Genre']=='Unknown'])
vgm_null_pub1 = pd.DataFrame(vgm[vgm['Publisher']=='Unknown'])

#**전처리 완료

#** Feature Engineering start

In [ ]:
vgm['Total_Sales'] = vgm['NA_Sales']+ vgm['EU_Sales']+ vgm['JP_Sales']+ vgm['Other_Sales']

In [ ]:
# 년도별로 정렬하고 결측치는 마지막에 덧붙인다
vgm.sort_values(by = 'Year', axis = 0, ascending=True, inplace=True)

vgm_year_0 = vgm[vgm['Year']==0]

vgm_year = vgm[vgm['Year'] != 0]

#결측치는 Unknown으로 설정하고 뒤에 덧붙인다


In [ ]:
#연도별 binning ==>> vgm_year 결측치는 없애고 연도별로 Binning하고 Feature 추가
min_year = vgm_year['Year'].min()
max_year = vgm_year['Year'].max()
bin_year = np.array([1980, 1990, 2000, 2005, 2010, 2015, 2020])
label_year = np.array(['~1990', '~2000', '~2005', '~2010', '~2015', '~2020'])
vgm_year['Period'] = pd.cut(vgm_year['Year'], bins = bin_year, labels= label_year, include_lowest=True)

# Year 결측치는 Unknown으로 범주 설정
vgm_year_0['Period'] = 'Unknown'

vgm = pd.concat([vgm_year, vgm_year_0])

vgm.reset_index(drop=True, inplace=True)

In [ ]:
vgm.groupby('Period').Total_Sales.sum()

In [ ]:
# plt.hist(vgm['Period'], bins=6)

In [ ]:
#vgm_gen_loc == 장르별 나라 판매량 / 나라별 장르 판매량

vgm_gen_loc = vgm.groupby('Genre').mean()
vgm_gen_loc = vgm_gen_loc.drop('Year', axis = 1)
# vgm_gen_loc = vgm_gen_loc.drop('Unknown', axis = 0)
vgm_gen_loc = vgm_gen_loc.drop('Total_Sales', axis = 1)
# vgm_gen_loc =  vgm_gen_loc.T
vgm_gen_loc1 = vgm_gen_loc.drop('Unknown', axis=0) 

In [ ]:
def gen_variance_loc(df, header) :
    # fig, axes = plt.subplots()
    colors = ['red', 'orange', 'yellow','dimgray','dimgray','dimgrey','darkgray','silver','lightgrey','lightgrey','lightgrey','lightgrey','lightgrey']
    plt.figure(figsize=(16,8), dpi=80)
    plt.bar(df.index, df[header])
    plt.title((header + ('\' Genre variance')))
    plt.show()
gen_variance_loc(vgm_gen_loc1, 'NA_Sales')
gen_variance_loc(vgm_gen_loc1, 'EU_Sales')
gen_variance_loc(vgm_gen_loc1, 'JP_Sales')
gen_variance_loc(vgm_gen_loc1, 'Other_Sales')


In [ ]:
# stats.chi2_contingency(vgm_gen_loc)
# stats.ttest_1samp(vgm_gen_loc['JP_Sales'], np.mean(vgm_gen_loc['JP_Sales']))
# scale = StandardScaler()
# vgm_gen_locZ = pd.DataFrame(scale.fit_transform(vgm_gen_loc.T))
# vgm_gen_locZ


In [ ]:
# North America에서 장르별 선호가 있는지?
p_gen_NA = stats.chisquare(vgm_gen_loc['NA_Sales']*1000) # P_value가 0.05 이하 ==> 선호가 있다!


#각 row를 가지고 분포 차이 ANOVA test == 각 나라별로 선호하는 장르의 차이가 있는지?
p_gen_loc = stats.f_oneway(vgm_gen_loc['NA_Sales'],vgm_gen_loc['EU_Sales'],vgm_gen_loc['JP_Sales'],vgm_gen_loc['Other_Sales']).pvalue #각 나라별 판매량의 분산이 서로 다르다!


# scaler = StandardScaler()
# vgm_gen_locZ = pd.DataFrame(scaler.fit_transform(vgm_gen_loc))
# vgm_gen_locZ
# newind = vgm_gen_loc.index
# newheader = vgm_gen_loc.columns
# vgm_gen_locZ.columns = newheader
# vgm_gen_locZ.index = newind
# # vgm_gen_loc.corr()

In [ ]:
# # vgm_gen_locZ
# p_gen_loc = stats.f_oneway(vgm_gen_locZ['NA_Sales'],vgm_gen_locZ['EU_Sales'],vgm_gen_locZ['JP_Sales'],vgm_gen_locZ['Other_Sales']).pvalue
p_gen_loc

##아래는 게임(이름)별 판매량 top~

In [ ]:
# 플랫폼 무시하고 이름으로 묶어서 전체기간 판매량 top == vgm_name_total


vgm_by_name_all = vgm.groupby(['Name','Genre','Publisher']).sum() #groupby에 'Period'를 넣으면 같은 게임이라도 다른 Period로 분류되는 게임들이 생겨서 값이 늘어남.
vgm_name_total = vgm_by_name_all.sort_values('Total_Sales', ascending=False)
vgm_name_total.drop('Year', axis=1, inplace=True)
vgm_name_total.reset_index(inplace=True)

In [ ]:
vgm_name_total

In [ ]:
# vgm_top_individual / 이름으로 안 묶었을 때(플랫폼 별도) 전체 기간 top

vgm_top_individual = vgm.sort_values('Total_Sales',ascending=False)
vgm_top_individual.reset_index(drop=True, inplace=True)
vgm_top_individual

#아래는 연도별 게임 판매량 1위

In [ ]:
#연도별 탑 게임 (플랫폼 별도)

vgm_period_total = pd.melt(vgm, id_vars = ['Period','Name','Platform','Genre','Publisher'], 
              value_vars = ['Total_Sales'],
               )
vgm_period_total.groupby('Period').max()



topgame_1990 = vgm_period_total[vgm_period_total['Period']=='~1990'].sort_values('value', ascending=False).head(1) 
topgame_2000 = vgm_period_total[vgm_period_total['Period']=='~2000'].sort_values('value', ascending=False).head(1) 
topgame_2005 = vgm_period_total[vgm_period_total['Period']=='~2005'].sort_values('value', ascending=False).head(1) 
topgame_2010 = vgm_period_total[vgm_period_total['Period']=='~2010'].sort_values('value', ascending=False).head(1) 
topgame_2015 = vgm_period_total[vgm_period_total['Period']=='~2015'].sort_values('value', ascending=False).head(1) 
topgame_2020 = vgm_period_total[vgm_period_total['Period']=='~2020'].sort_values('value', ascending=False).head(1) 

topgame_unknown = vgm_period_total[vgm_period_total['Period']=='Unknown'].sort_values('value', ascending=False).head(1) 


In [ ]:
vgm_period_total

In [ ]:
vgm_period_total.sort_values('value', ascending=False)

In [ ]:
#  연도별 판매량 탑 리스트

vgm_1990_total = vgm_period_total[vgm_period_total['Period']=='~1990'].sort_values('value', ascending=False).reset_index(drop=True)
vgm_2000_total = vgm_period_total[vgm_period_total['Period']=='~2000'].sort_values('value', ascending=False).reset_index(drop=True)
vgm_2005_total = vgm_period_total[vgm_period_total['Period']=='~2005'].sort_values('value', ascending=False).reset_index(drop=True)
vgm_2010_total = vgm_period_total[vgm_period_total['Period']=='~2010'].sort_values('value', ascending=False).reset_index(drop=True)
vgm_2015_total = vgm_period_total[vgm_period_total['Period']=='~2015'].sort_values('value', ascending=False).reset_index(drop=True)
vgm_2020_total = vgm_period_total[vgm_period_total['Period']=='~2020'].sort_values('value', ascending=False).reset_index(drop=True)




In [ ]:
vgm_1990_total

#아래는 연도별 장르 트렌드 (장르별 출시게임 수)

In [ ]:
vgm_period_gen = pd.melt(vgm, id_vars = ['Period','Name'], 
              value_vars = ['Genre'], var_name='Genre', value_name='genre_name'
               )
vgm_period_gen1 = pd.DataFrame(vgm_period_gen.groupby('Period').genre_name.value_counts())
pd.set_option('display.max_rows', 100)

vgm_period_gen1.rename(columns={'genre_name' : 'numbers' }, inplace=True)

vgm_period_gen1.reset_index(inplace=True)

vgm_gen_1990 = vgm_period_gen1[vgm_period_gen1['Period']=='~1990'].drop('Period', axis=1)
vgm_gen_2000 = vgm_period_gen1[vgm_period_gen1['Period']=='~2000'].drop('Period', axis=1)
vgm_gen_2005 = vgm_period_gen1[vgm_period_gen1['Period']=='~2005'].drop('Period', axis=1)
vgm_gen_2010 = vgm_period_gen1[vgm_period_gen1['Period']=='~2010'].drop('Period', axis=1)
vgm_gen_2015 = vgm_period_gen1[vgm_period_gen1['Period']=='~2015'].drop('Period', axis=1)
vgm_gen_2020 = vgm_period_gen1[vgm_period_gen1['Period']=='~2020'].drop('Period', axis=1)
vgm_gen_unknown = vgm_period_gen1[vgm_period_gen1['Period']=='Unknown'].drop('Period', axis=1)

vgm_gen_1990
# plt.figure(figsize=(16,8), dpi=80)
# plt.bar(vgm_gen_1990['genre_name'], vgm_gen_1990['numbers'])
# colors = ['black','dimgray','dimgrey','darkgray','silver','lightgrey']
def period_gen_plot(df, year) :
    # fig, axes = plt.subplots()
    colors = ['red', 'orange', 'yellow','dimgray','dimgray','dimgrey','darkgray','silver','lightgrey','lightgrey','lightgrey','lightgrey','lightgrey']
    plt.figure(figsize=(16,8), dpi=80)
    plt.bar(df['genre_name'], df['numbers'], color = colors)
    if year == 'Unknown' :
        plt.title('Trend in Genre for Unknown period', fontsize = 35)
    else : 
        plt.title('~' + str(year)+'\'s trend in game genre', fontsize = 35)
    plt.show()
# #연도별 장르 수 그래프 그리기    
# period_gen_plot(vgm_gen_1990,1990)
# period_gen_plot(vgm_gen_2000,2000)
# period_gen_plot(vgm_gen_2005,2005)
# period_gen_plot(vgm_gen_2010,2010)
# period_gen_plot(vgm_gen_2015,2015)
# period_gen_plot(vgm_gen_2020,2020)
# period_gen_plot(vgm_gen_unknown,'Unknown')


In [ ]:
vgm_gen_2020

In [ ]:
# stats.f_oneway(vgm_gen_2020,vgm_gen_2015,vgm_gen_2010,vgm_gen_2005,vgm_gen_2000,vgm_gen_1990)

# 각 연도별 출시량을 통한 선호도가 있나?
stats.chisquare(vgm_gen_2020['numbers']).pvalue
def trend_chi(x) :
    y = stats.chisquare(x).pvalue
    if y < 0.05 :
        print('There is something. P_value : ', y)
    else : 
        print('No remarkable differences')
trend_chi(vgm_gen_2020['numbers'])


#연도별 장르의 판매량 확인 필요below

In [ ]:
# vgm_period_gen2 = pd.melt(vgm_year, id_vars = ['Period','Genre'], 
#               value_vars = [''], var_name='Genre', value_name='genre_name'
#                )
# vgm_period_gen2

vgm_period_genS = vgm.groupby('Period').Genre.value_counts()
vgm_period_genS
vgm_gen_total = pd.DataFrame(vgm.groupby(['Period','Genre']).Total_Sales.sum())
vgm_gen_total.reset_index(inplace=True)

def period_gen_sales_plot(df, year) :
    # fig, axes = plt.subplots()
    colors = ['red', 'orange', 'yellow','dimgray','dimgray','dimgrey','darkgray','silver','lightgrey','lightgrey','lightgrey','lightgrey','lightgrey']
    plt.figure(figsize=(16,8), dpi=80)
    # plt.ylim(0,15)
    plt.bar(df['Genre'], df['Total_Sales'])
    plt.title('~' + str(year)+'\'s Total sales by game genre', fontsize = 35)
    plt.show()

vgm_gen_total1990 = vgm_gen_total[vgm_gen_total['Period']=='~1990'].drop('Period', axis=1)
vgm_gen_total2000 = vgm_gen_total[vgm_gen_total['Period']=='~2000'].drop('Period', axis=1)
vgm_gen_total2005 = vgm_gen_total[vgm_gen_total['Period']=='~2005'].drop('Period', axis=1)
vgm_gen_total2010 = vgm_gen_total[vgm_gen_total['Period']=='~2010'].drop('Period', axis=1)
vgm_gen_total2015 = vgm_gen_total[vgm_gen_total['Period']=='~2015'].drop('Period', axis=1)
vgm_gen_total2020 = vgm_gen_total[vgm_gen_total['Period']=='~2020'].drop('Period', axis=1)
vgm_gen_total_unknown = vgm_gen_total[vgm_gen_total['Period']=='Unknown'].drop('Period', axis=1)


# vgm_gen_total2015


In [ ]:
# #연도별 장르별 게임 수

# period_gen_plot(vgm_gen_1990,1990)
# period_gen_plot(vgm_gen_2000,2000)
# period_gen_plot(vgm_gen_2005,2005)
# period_gen_plot(vgm_gen_2010,2010)
# period_gen_plot(vgm_gen_2015,2015)
# period_gen_plot(vgm_gen_2020,2020)
# period_gen_plot(vgm_gen_unknown,'Unknown')


# #연도별 장르별 게임 실제 판매량 그래프

# period_gen_sales_plot(vgm_gen_total1990, 1990)
# period_gen_sales_plot(vgm_gen_total2000, 2000)
# period_gen_sales_plot(vgm_gen_total2005, 2005)
# period_gen_sales_plot(vgm_gen_total2010, 2010)
# period_gen_sales_plot(vgm_gen_total2015, 2015)
# period_gen_sales_plot(vgm_gen_total2020, 2020)
# period_gen_sales_plot(vgm_gen_total_unknown, 'Unknown')




In [ ]:
#Unknown 장르의 판매량
vgm_unknown_sales = vgm[vgm['Genre'] == 'Unknown'].sort_values('Total_Sales', ascending=False)
vgm_unknown_worth = vgm_unknown_sales[vgm_unknown_sales['Total_Sales'] > 0.99]

vgm_unknown_worth.reset_index(drop=True, inplace=True)
vgm_unknown = vgm[vgm['Genre'] == 'Unknown']
vgm_unknown = vgm_unknown.groupby('Year').Total_Sales.sum()
vgm_unknown

In [ ]:
vgm_total_per_gen1990 = vgm_gen_1990.copy()
vgm_total_per_gen1990['Total_Sales'] = vgm_gen_total1990['Total_Sales']
vgm_total_per_gen1990['Sales_per_Game'] = vgm_total_per_gen1990['Total_Sales'] / vgm_total_per_gen1990['numbers']
vgm_total_per_gen1990

vgm_total_per_gen2000 = vgm_gen_2000.copy()
vgm_total_per_gen2000['Total_Sales'] = vgm_gen_total2000['Total_Sales']
vgm_total_per_gen2000['Sales_per_Game'] = vgm_total_per_gen2000['Total_Sales'] / vgm_total_per_gen2000['numbers']
vgm_total_per_gen2000

vgm_total_per_gen2005 = vgm_gen_2005.copy()
vgm_total_per_gen2005['Total_Sales'] = vgm_gen_total2005['Total_Sales']
vgm_total_per_gen2005['Sales_per_Game'] = vgm_total_per_gen2005['Total_Sales'] / vgm_total_per_gen2005['numbers']
vgm_total_per_gen2005

vgm_total_per_gen2010 = vgm_gen_2010.copy()
vgm_total_per_gen2010['Total_Sales'] = vgm_gen_total2010['Total_Sales']
vgm_total_per_gen2010['Sales_per_Game'] = vgm_total_per_gen2010['Total_Sales'] / vgm_total_per_gen2010['numbers']
vgm_total_per_gen2010

vgm_total_per_gen2015 = vgm_gen_2015.copy()
vgm_total_per_gen2015['Total_Sales'] = vgm_gen_total2015['Total_Sales']
vgm_total_per_gen2015['Sales_per_Game'] = vgm_total_per_gen2015['Total_Sales'] / vgm_total_per_gen2015['numbers']
vgm_total_per_gen2015

vgm_total_per_gen2020 = vgm_gen_2020.copy()
vgm_total_per_gen2020['Total_Sales'] = vgm_gen_total2020['Total_Sales']
vgm_total_per_gen2020['Sales_per_Game'] = vgm_total_per_gen2020['Total_Sales'] / vgm_total_per_gen2020['numbers']
vgm_total_per_gen2020

vgm_total_per_gen_unknown = vgm_gen_unknown.copy()
vgm_total_per_gen_unknown['Total_Sales'] = vgm_gen_total_unknown['Total_Sales']
vgm_total_per_gen_unknown['Sales_per_Game'] = vgm_total_per_gen_unknown['Total_Sales'] / vgm_total_per_gen_unknown['numbers']
vgm_total_per_gen_unknown

vgm_total_per_gen2020 = vgm_gen_2020.copy()
vgm_total_per_gen2020['Total_Sales'] = vgm_gen_total2020['Total_Sales']
vgm_total_per_gen2020['Sales_per_Game'] = vgm_total_per_gen2020['Total_Sales'] / vgm_total_per_gen2020['numbers']
vgm_total_per_gen2020['Sales_per_Game']

In [ ]:
def sales_per_gen_plot(df, year) :
    # fig, axes = plt.subplots()
    colors = ['red', 'orange', 'yellow','dimgray','dimgray','dimgrey','darkgray','silver','lightgrey','lightgrey','lightgrey','lightgrey','lightgrey']
    plt.figure(figsize=(16,8), dpi=80)
    # plt.ylim(0,15)
    plt.bar(df['genre_name'], df['Sales_per_Game'])
    plt.title('~' + str(year)+'\'s Sale-per-Game in Genre', fontsize = 35)
    plt.show()

# sales_per_gen_plot(vgm_total_per_gen1990,1990)
# sales_per_gen_plot(vgm_total_per_gen2000,2000)
# sales_per_gen_plot(vgm_total_per_gen2005,2005)
# sales_per_gen_plot(vgm_total_per_gen2010,2010)
# sales_per_gen_plot(vgm_total_per_gen2015,2015)
# sales_per_gen_plot(vgm_total_per_gen2020,2020)
# sales_per_gen_plot(vgm_total_per_gen_unknown,'Unknown')

In [ ]:

trend_chi(vgm_total_per_gen2020['Sales_per_Game']*1000)

In [ ]:
period_gen_plot(vgm_gen_1990,1990)
period_gen_sales_plot(vgm_gen_total1990, 1990)
sales_per_gen_plot(vgm_total_per_gen1990,1990)



period_gen_plot(vgm_gen_2000,2000)
period_gen_sales_plot(vgm_gen_total2000, 2000)
sales_per_gen_plot(vgm_total_per_gen2000,2000)

period_gen_plot(vgm_gen_2005,2005)
period_gen_sales_plot(vgm_gen_total2005, 2005)
sales_per_gen_plot(vgm_total_per_gen2005,2005)

period_gen_plot(vgm_gen_2010,2010)
period_gen_sales_plot(vgm_gen_total2010, 2010)
sales_per_gen_plot(vgm_total_per_gen2010,2010)

period_gen_plot(vgm_gen_2015,2015)
period_gen_sales_plot(vgm_gen_total2015, 2015)
sales_per_gen_plot(vgm_total_per_gen2015,2015)


period_gen_plot(vgm_gen_2020,2020)
period_gen_sales_plot(vgm_gen_total2020, 2020)
sales_per_gen_plot(vgm_total_per_gen2020,2020)


period_gen_plot(vgm_gen_unknown,'Unknown')
period_gen_sales_plot(vgm_gen_total_unknown, 'Unknown')
sales_per_gen_plot(vgm_total_per_gen_unknown,'Unknown')



In [ ]:
vgm_total_per_gen2015

In [ ]:
vgm_2015_total[vgm_2015_total['Genre']=='Unknown']

In [ ]:
#장르별 게임 하나당 이익률 == vgm_gen_count

vgm_gen = pd.melt(vgm, id_vars = ['Period','Name'], 
              value_vars = ['Genre'], var_name='Genre', value_name='genre_name'
               )
vgm_gen1 = pd.DataFrame(vgm_gen.groupby('Period').genre_name.value_counts())
# pd.set_option('display.max_rows', 100)

vgm_gen1.rename(columns={'genre_name' : 'numbers' }, inplace=True)

vgm_gen1.reset_index(inplace=True)
vgm_gen_count = pd.DataFrame(vgm_gen1.groupby('genre_name').numbers.sum())
vgm_gen_count

vgm_total_per_gen = pd.DataFrame(vgm.groupby('Genre').sum()['Total_Sales'])

vgm_gen_count['Total_Sales'] = vgm_total_per_gen['Total_Sales']
vgm_gen_count['Sales_per_Game'] = vgm.groupby('Genre').Total_Sales.mean()


vgm_gen_count

In [ ]:
vgm.groupby('Genre').Total_Sales.mean()

In [ ]:
vgm_gen_loc = vgm.groupby('Genre')[['NA_Sales','EU_Sales','JP_Sales','Other_Sales', 'Total_Sales']].mean()
# stats.chisquare(vgm_gen_loc['Other_Sales'])

def gen_loc_trend(x, header) :
    y = stats.chisquare(x * 1000).pvalue
    print(header)
    if y < 0.05 :
        print('There is something. P-value is : ', y)
    else : 
        print('Nothing remarkable. P-value is : ', y)

gen_loc_trend(vgm_gen_loc["NA_Sales"], "NA_Sales")
gen_loc_trend(vgm_gen_loc["EU_Sales"], "EU_Sales")
gen_loc_trend(vgm_gen_loc["JP_Sales"], "JP_Sales")
gen_loc_trend(vgm_gen_loc["Other_Sales"], "Other_Sales")
vgm_gen_loc["NA_Sales"].sort_values(ascending=False)

In [ ]:
vgm_gen_num = pd.DataFrame(vgm.groupby(['Period']).Genre.value_counts())
vgm_gen_num_total = pd.DataFrame(vgm.groupby(['Period','Genre']).Total_Sales.sum())
# newcol = vgm_gen_num_total['Total_Sales']
# # vgm_gen_num['Total_Sales'] = newcol
vgm_gen_num.rename(columns = {'Genre' : 'Numbers'}, inplace=True)

# vgm_gen_num.reset_index(inplace=True)



In [ ]:
vgm_gen_num
vgm_gen_num_total
vgm_gen_num1 = vgm_gen_num.merge(vgm_gen_num_total,right_index=True,left_index=True)
# vgm_gen_num_total

# vgm_gen_numUN = vgm_gen_num[vgm_gen_num['Period']=='Unknown'].sort_values('Genre')
# vgm_gen_num1990 = vgm_gen_num[vgm_gen_num['Period']=='~1990'].sort_values('Genre')
# vgm_gen_num2000 = vgm_gen_num[vgm_gen_num['Period']=='~2000'].sort_values('Genre')
# vgm_gen_num2005 = vgm_gen_num[vgm_gen_num['Period']=='~2005'].sort_values('Genre')
# vgm_gen_num2010 = vgm_gen_num[vgm_gen_num['Period']=='~2010'].sort_values('Genre')
# vgm_gen_num2015 = vgm_gen_num[vgm_gen_num['Period']=='~2015'].sort_values('Genre')
# vgm_gen_num2020 = vgm_gen_num[vgm_gen_num['Period']=='~2020'].sort_values('Genre')
vgm_gen_num1.reset_index(inplace=True)

In [ ]:
vgm_action = vgm_gen_num1[vgm_gen_num1['Genre']=='Action'].reset_index(drop=True)
vgm_sports = vgm_gen_num1[vgm_gen_num1['Genre']=='Sports'].reset_index(drop=True)
vgm_misc = vgm_gen_num1[vgm_gen_num1['Genre']=='Misc'].reset_index(drop=True)
vgm_racing = vgm_gen_num1[vgm_gen_num1['Genre']=='Racing'].reset_index(drop=True)
vgm_shooter = vgm_gen_num1[vgm_gen_num1['Genre']=='Shooter'].reset_index(drop=True)
vgm_role = vgm_gen_num1[vgm_gen_num1['Genre']=='Role-Playing'].reset_index(drop=True)
vgm_simulation = vgm_gen_num1[vgm_gen_num1['Genre']=='Simulation'].reset_index(drop=True)
vgm_platform = vgm_gen_num1[vgm_gen_num1['Genre']=='Platform'].reset_index(drop=True)
vgm_puzzle = vgm_gen_num1[vgm_gen_num1['Genre']=='Puzzle'].reset_index(drop=True)
vgm_adventure = vgm_gen_num1[vgm_gen_num1['Genre']=='Adventure'].reset_index(drop=True)
vgm_fighting = vgm_gen_num1[vgm_gen_num1['Genre']=='Fighting'].reset_index(drop=True)
vgm_strategy = vgm_gen_num1[vgm_gen_num1['Genre']=='Strategy'].reset_index(drop=True)
vgm_gen_unknown = vgm_gen_num1[vgm_gen_num1['Genre']=='Unknown'].reset_index(drop=True)

In [ ]:
vgm_action['Sales_per_Game'] = vgm_action['Total_Sales']/ vgm_action['Numbers']
vgm_misc['Sales_per_Game']= vgm_misc['Total_Sales'] / vgm_misc['Numbers']
vgm_sports['Sales_per_Game']= vgm_sports['Total_Sales'] / vgm_sports['Numbers']
vgm_racing['Sales_per_Game']= vgm_racing['Total_Sales'] / vgm_racing['Numbers']
vgm_shooter['Sales_per_Game']= vgm_shooter['Total_Sales'] / vgm_shooter['Numbers']
vgm_role['Sales_per_Game']= vgm_role['Total_Sales'] / vgm_role['Numbers']
vgm_simulation['Sales_per_Game']= vgm_simulation['Total_Sales'] / vgm_simulation['Numbers']
vgm_platform['Sales_per_Game']= vgm_platform['Total_Sales'] / vgm_platform['Numbers']
vgm_puzzle['Sales_per_Game']= vgm_puzzle['Total_Sales'] / vgm_puzzle['Numbers']
vgm_adventure['Sales_per_Game']= vgm_adventure['Total_Sales'] / vgm_adventure['Numbers']
vgm_fighting['Sales_per_Game']= vgm_fighting['Total_Sales'] / vgm_fighting['Numbers']
vgm_strategy['Sales_per_Game']= vgm_strategy['Total_Sales'] / vgm_strategy['Numbers']
vgm_gen_unknown['Sales_per_Game']= vgm_gen_unknown['Total_Sales'] / vgm_gen_unknown['Numbers']


In [ ]:
vgm_action

In [ ]:
vgm_gen_mean = pd.DataFrame(
                            [vgm_action.mean(),
                             vgm_misc.mean(),
                             vgm_sports.mean(),
                             vgm_racing.mean(),
                             vgm_shooter.mean(),
                             vgm_role.mean(),
                             vgm_simulation.mean(),
                             vgm_platform.mean(),
                             vgm_puzzle.mean(),
                             vgm_fighting.mean(),
                             vgm_strategy.mean(),
                             vgm_gen_unknown.mean(),
                             ], 
                            index = ['Action', 'Misc', 'Sports', 'Racing', 'Shooter', 
                                     'Role-Playing', 'Simulation', 'Platform', 'Puzzle',
                                     'Figting', 'Strategy', 'Unknown'],
                        
)

In [ ]:
# vgm_gen_num2005
vgm_gen_mean

In [ ]:
vgm_top_individual.head(10)

In [ ]:
vgm_name_total.head(10)
vgm_name_total[vgm_name_total['Publisher'] == 'Nintendo'].value_counts().sum()

In [ ]:
#2010년부터 출시된 게임은 JP_Sales가 의미있진 않음. 

vgm_later_2009 = vgm[vgm['Year'] > 2009].sort_values('Total_Sales', ascending=False)
vgm_from_2010 = vgm_later_2009.groupby(['Name','Genre','Publisher'])[['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Total_Sales']].sum().sort_values('Total_Sales', ascending=False)
vgm_from_2010[vgm_from_2010['Total_Sales'] > 1.0]

In [ ]:
vgm_from_2010top = vgm_from_2010[vgm_from_2010['Total_Sales'] > 1.0]
vgm_from_2010top.reset_index(inplace=True)
vgm_from_2010top

In [ ]:
# 2010년 이후 게임 중 판매량 1M 이상, 장르 개수
vgm_from_2010top['Genre'].value_counts()

In [ ]:
#Unknown == 포켓몬이랑 Wii party
vgm_from_2010top.groupby('Genre').Total_Sales.mean()


In [ ]:
#2010년부터 출시된 게임 중 일본 판매량이 전체 판매량의 5%를 넘는 게임 == 1675개 
vgm_JP = vgm_from_2010[(vgm_from_2010['JP_Sales'] / vgm_from_2010['Total_Sales']) > 0.05]
#vgm_JP 중에서 NA, EU, Other 판매량이 전체 판매량의 5%를 넘는 게임은 517개 뿐
vgm_JP[(vgm_JP['NA_Sales'] +vgm_JP['EU_Sales'] + vgm_JP['Other_Sales'])/ vgm_JP['Total_Sales'] > 0.05 ]

In [ ]:
vgm_tmp = vgm[vgm['Total_Sales'] > 1.0]
vgm_tmp
# vgm_tmp[(vgm_tmp['JP_Sales'] / vgm_tmp['Total_Sales']) > 0.05]

In [ ]:
#특히, 전체 판매량이 1M을 넘는 게임 중 일본 판매량이 5%이상인 경우는 158개 뿐임.
vgm_JPtop = vgm_from_2010top[(vgm_from_2010top['JP_Sales'] / vgm_from_2010top['Total_Sales']) > 0.05]
vgm_JPtop

In [ ]:
#이름으로 묶은 데이터프레임을 갖다대면 일본 판매량의 비율을 보여준다
def percent_JP(df,index_num):
    fig = plt.figure(figsize=(9,9))
    fig.set_facecolor('white')

    ax = fig.add_subplot()
    ax.pie(x=df.iloc[index_num][['NA_Sales','EU_Sales','JP_Sales','Other_Sales']], 
       labels= ['NA_Sales','EU_Sales','JP_Sales','Other_Sales'],
       autopct=lambda p : '{:.2f}%'.format(p))
    plt.title(df.index[index_num][0])
    plt.show()
percent_JP(vgm_from_2010,0)

In [ ]:
#그러나, 판매량이 1M을 넘는 게임들 중 Other_Sales의 비중이 5%가 넘는 게임은 502개.
vgm_others = vgm_from_2010top[(vgm_from_2010top['Other_Sales'] / vgm_from_2010top['Total_Sales']) > 0.05]
vgm_others

In [ ]:
vgm_noJP = vgm[(vgm['Publisher'] != 'Nintendo') & (vgm['Year'] > 2009)]
vgm_noJP

vgm_noJP_gen_loc = vgm_noJP.groupby('Genre').mean()
vgm_noJP_gen_loc = vgm_noJP_gen_loc.drop('Year', axis = 1)
# vgm_gen_loc = vgm_gen_loc.drop('Unknown', axis = 0)
vgm_noJP_gen_loc = vgm_noJP_gen_loc.drop('Total_Sales', axis = 1)
# vgm_gen_loc =  vgm_gen_loc.T
vgm_noJP_gen_loc1 = vgm_noJP_gen_loc.drop('Unknown', axis=0) 

gen_variance_loc(vgm_noJP_gen_loc1, 'NA_Sales')
gen_variance_loc(vgm_noJP_gen_loc1, 'EU_Sales')
gen_variance_loc(vgm_noJP_gen_loc1, 'JP_Sales')
gen_variance_loc(vgm_noJP_gen_loc1, 'Other_Sales')

In [ ]:
vgm_noJP[vgm_noJP['Genre'] == 'Simulation'].sort_values('Total_Sales', ascending=False)

vgm[vgm['Genre'] == 'Simulation'].sort_values('Total_Sales', ascending=False).head(50)


In [ ]:
vgm_noJP_shooter = vgm_noJP[vgm_noJP['Genre'] == 'Shooter'].sort_values('Total_Sales', ascending=False)
vgm_noJP_shooter.describe() # 중간 값 0.40

vgm_noJP_Action = vgm_noJP[vgm_noJP['Genre'] == 'Action'].sort_values('Total_Sales', ascending=False)
vgm_noJP_Action.describe()

# vgm_noJP_Platform = vgm_noJP[vgm_noJP['Genre'] == 'Platform'].sort_values('Total_Sales', ascending=False)
# vgm_noJP_Platform.describe()


# vgm_noJP_Simulation = vgm_noJP[vgm_noJP['Genre'] == 'Simulation'].sort_values('Total_Sales', ascending=False)
# vgm_noJP_Simulation.describe()


# vgm_noJP_Strategy = vgm_noJP[vgm_noJP['Genre'] == 'Strategy'].sort_values('Total_Sales', ascending=False)
# vgm_noJP_Strategy.describe()


vgm_noJP_Fighting = vgm_noJP[vgm_noJP['Genre'] == 'Fighting'].sort_values('Total_Sales', ascending=False)
vgm_noJP_Fighting.describe()

# vgm_noJP_Adventure = vgm_noJP[vgm_noJP['Genre'] == 'Adventure'].sort_values('Total_Sales', ascending=False)
# vgm_noJP_Adventure.describe()

# vgm_noJP_Role_Playing = vgm_noJP[vgm_noJP['Genre'] == 'Role-Playing'].sort_values('Total_Sales', ascending=False)
# vgm_noJP_Role_Playing.describe()

# vgm_noJP_Puzzle = vgm_noJP[vgm_noJP['Genre'] == 'Puzzle'].sort_values('Total_Sales', ascending=False)
# vgm_noJP_Puzzle.describe()

# vgm_noJP_Racing = vgm_noJP[vgm_noJP['Genre'] == 'Racing'].sort_values('Total_Sales', ascending=False)
# vgm_noJP_Racing.describe()

# vgm_noJP_Sports = vgm_noJP[vgm_noJP['Genre'] == 'Sports'].sort_values('Total_Sales', ascending=False)
# vgm_noJP_Sports.describe()

# vgm_noJP_Misc = vgm_noJP[vgm_noJP['Genre'] == 'Misc'].sort_values('Total_Sales', ascending=False)
# vgm_noJP_Misc.describe()

